### Importing all the libraries

## Installing all the packages required

## Importing all the libraries

In [2]:
import pandas as pd
import requests
import numpy as np
import folium
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup

# #Getting table from Wikipedia

In [109]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())

My_table = soup.find('table',{'class':'wikitable sortable'})


links = My_table.findAll('tr')[1:]


link2 = My_table.findAll('tr')[1:]

link3 = My_table.findAll('tr')[1:]
Postcode = []
Borough = []
Neighborhood = []
for link in links:
    x = link.find_all('td')[0]
    y = link.find_all('td')[2]
    Postcode.append(x.text)
    Borough.append(y.text)
    z = link.find_all('td')[1]
    Neighborhood.append(z.text)

##    y = link.find_all('td')[1]
#    Borough.append(y.text)
    
#for link in link3:
#    z = link.find_all('a')
#    Neighborhood.append(z.text)
    


## Creating a data frame from the table

In [110]:
df = pd.DataFrame(columns=['Postcode','Borough','Neighborhood'])

## Filling the pandas data frame

In [111]:
df['Postcode'] = Postcode
df['Borough'] = Neighborhood
df['Neighborhood'] = Borough

## cleaning the neighborhood coloumn

In [112]:
x = df['Neighborhood']

In [113]:
y = list(x)

In [114]:
z = [col.split('\n')[0] for col in y]

In [115]:
df['Neighborhood'] = z

## Removing all NaN values

In [116]:
df.replace('Not assigned','NaN', inplace = True)


In [117]:
df.head()

,Postcode,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [118]:
borough_not_required = df[df['Borough']== 'NaN'].index.tolist()

In [119]:
df = df.drop(borough_not_required).reset_index(drop=True)

In [120]:
df.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


## Joining the Neighborhood coloumn for the same Postcode

In [121]:
x  = df.groupby(['Postcode','Borough'])['Neighborhood'].apply(','.join)

In [122]:
toronto = pd.DataFrame(x) 

In [123]:
toronto.reset_index(inplace=True)

In [124]:
toronto.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [125]:
toronto.shape

(103, 3)

## Task 2

In [126]:
!conda install -c conda-forge geocoder

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geocoder                  1.38.1                     py_0    conda-forge


In [127]:
import geocoder

## Defining function for getting latitude longitude for postal code

In [128]:
def getlatlng(postal_code):
    lat_lng_coords = None
    while (lat_lng_coords == None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        return lat_lng_coords

In [129]:
getlatlng('M1B')

[43.811525000000074, -79.19551721399995]

In [130]:
coords = [getlatlng(x) for x in toronto['Postcode'].tolist()]

## Adding coordinates to main dataframe i.e. toronto

In [131]:
toronto['Latitude']=[coords[i][0] for i in range(len(coords))]
toronto['Longitude'] = [coords[i][1] for i in range(len(coords))] 

In [132]:
toronto.head(10)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.726245,-79.263670
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.713133,-79.285055
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.723575,-79.234976
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.696665,-79.260163


In [133]:
toronto[toronto['Postcode'] == 'M5G']

,Postcode,Borough,Neighborhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.656091,-79.38493


## Task 3

In [134]:
from sklearn.cluster import KMeans

In [135]:
toronto.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


## SEecting Downtown Toronto borough

In [136]:
downtown_toronto = toronto[toronto['Borough']=="Downtown Toronto"]

In [137]:
downtown_toronto.reset_index(drop=True,inplace=True)

In [138]:
downtown_toronto.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.681940,-79.378474
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.668160,-79.366602
2,M4Y,Downtown Toronto,Church and Wellesley,43.666585,-79.381302
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.655120,-79.362640
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657363,-79.378180


In [139]:
adress = 'Downtown Toronto'
x = Nominatim(user_agent="kartik")
geo = x.geocode(adress)

In [140]:
down_lat = geo.latitude
down_lng = geo.longitude

In [141]:
down_lat,down_lng

(43.6541737, -79.3808116451341)

In [142]:
down_map =  folium.Map([down_lat,down_lng], zoom_start = 12)

In [143]:
down_map

In [144]:
for lat , lng , neighborhood in zip(downtown_toronto['Latitude'],downtown_toronto['Longitude'],downtown_toronto['Neighborhood']):
    label = folium.Popup(neighborhood, parse_html=True)
    folium.CircleMarker([lat, lng],
                        radius=5,
                        popup=label,
                        color='blue',
                        fill=True,
                        fill_color='#3186cc',
                        fill_opacity=0.7,
                        parse_html=False).add_to(down_map)

In [145]:
CLIENT_ID = 'UUDATNQLSPQ3VLPACBR2Q2Q0BFDCM0ZXNM0D4T24R2TKN2WS' # your Foursquare ID
CLIENT_SECRET = 'ISR2D5ZTUDEZ5FHP3OEZNFB4HJCHUBFTDOVE5KHKL2OALHID' # your Foursquare Secret
VERSION = '20190421' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UUDATNQLSPQ3VLPACBR2Q2Q0BFDCM0ZXNM0D4T24R2TKN2WS
CLIENT_SECRET:ISR2D5ZTUDEZ5FHP3OEZNFB4HJCHUBFTDOVE5KHKL2OALHID


In [146]:
downtown_toronto.loc[0,'Neighborhood']

'Rosedale'

## lets get top 100 venue near Rosedale

In [147]:
neighborhood_lat = downtown_toronto.loc[0,'Latitude']
neighborhood_lng = downtown_toronto.loc[0,'Longitude']


In [148]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_lat, 
    neighborhood_lng, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=UUDATNQLSPQ3VLPACBR2Q2Q0BFDCM0ZXNM0D4T24R2TKN2WS&client_secret=ISR2D5ZTUDEZ5FHP3OEZNFB4HJCHUBFTDOVE5KHKL2OALHID&v=20190421&ll=43.681940000000054,-79.37847416699998&radius=500&limit=100'

In [149]:
results = requests.get(url).json()

In [150]:

venue =results['response']['groups'][0]['items']

In [151]:
from pandas.io.json import json_normalize

In [152]:
nearby_venue = json_normalize(venue)

In [153]:
nearby_venue['venue.categories'][0][0].keys()

dict_keys(['shortName', 'name', 'id', 'icon', 'primary', 'pluralName'])

In [154]:
def get_category(row):
    try:
        category_list = row['categories']
    except:
        category_list = row['venue.categories']
    if len(category_list) == 0:
        return None
    else:
        return category_list[0]['name']

In [155]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venue =nearby_venue.loc[:, filtered_columns]

In [156]:
nearby_venue

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,TD Canada Trust - Closed,"[{'shortName': 'Bank', 'name': 'Bank', 'id': '...",43.686094,-79.376549
1,Rosedale Park,"[{'shortName': 'Playground', 'name': 'Playgrou...",43.682328,-79.378934
2,Rosedale Tennis Club,"[{'shortName': 'Tennis Court', 'name': 'Tennis...",43.683226,-79.378984
3,Whitney Park,"[{'shortName': 'Park', 'name': 'Park', 'id': '...",43.682036,-79.373788
4,Mooredale House,"[{'shortName': 'Building', 'name': 'Building',...",43.678631,-79.380091


In [157]:
nearby_venue['venue.categories'] = nearby_venue.apply(get_category, axis=1)

In [158]:
nearby_venue

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,TD Canada Trust - Closed,Bank,43.686094,-79.376549
1,Rosedale Park,Playground,43.682328,-79.378934
2,Rosedale Tennis Club,Tennis Court,43.683226,-79.378984
3,Whitney Park,Park,43.682036,-79.373788
4,Mooredale House,Building,43.678631,-79.380091


In [159]:

downtown_toronto.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.681940,-79.378474
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.668160,-79.366602
2,M4Y,Downtown Toronto,Church and Wellesley,43.666585,-79.381302
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.655120,-79.362640
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657363,-79.378180


## Explore all neighborhood near Downtown

In [160]:
def get_nearby_venue(names, latitudes, longitudes, radius=500):
    venue_list = []
    for name, lat , lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venue_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']
        ) for v in results])
    
    nearby_venues = pd.DataFrame([cols for x in venue_list for  cols in x])
    nearby_venues.columns = ['Neighborhood' , 'Neighborhood Latitude',
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)
        

In [161]:
downtown_venues = get_nearby_venue(downtown_toronto['Neighborhood'] , downtown_toronto['Latitude'] , downtown_toronto['Longitude'])

Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie


In [162]:
downtown_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.68194,-79.378474,TD Canada Trust - Closed,43.686094,-79.376549,Bank
1,Rosedale,43.68194,-79.378474,Rosedale Park,43.682328,-79.378934,Playground
2,Rosedale,43.68194,-79.378474,Rosedale Tennis Club,43.683226,-79.378984,Tennis Court
3,Rosedale,43.68194,-79.378474,Whitney Park,43.682036,-79.373788,Park
4,Rosedale,43.68194,-79.378474,Mooredale House,43.678631,-79.380091,Building


## Analyze each neighborhood

In [164]:
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

In [166]:
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood']

In [167]:
fixed_col = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])

In [168]:
downtown_onehot = downtown_onehot[fixed_col]

In [191]:
downtown_onehot.head()

,Adult Boutique,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [171]:
downtown_onehot.shape

(1252, 184)

## Merge the details

In [172]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()

In [192]:
downtown_grouped.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide,King,Richmond",0.0,0.0,0.030000,0.010000,0.00,0.0,0.030000,0.0,0.000000,...,0.0,0.0,0.00,0.0,0.010000,0.000000,0.00,0.00,0.01,0.0
1,Berczy Park,0.0,0.0,0.000000,0.016129,0.00,0.0,0.000000,0.0,0.016129,...,0.0,0.0,0.00,0.0,0.016129,0.000000,0.00,0.00,0.00,0.0
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.0,0.0,0.014706,0.000000,0.00,0.0,0.014706,0.0,0.000000,...,0.0,0.0,0.00,0.0,0.000000,0.014706,0.00,0.00,0.00,0.0
3,"Cabbagetown,St. James Town",0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.00,0.0,0.000000,0.000000,0.00,0.00,0.00,0.0
4,Central Bay Street,0.0,0.0,0.010000,0.010000,0.01,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.01,0.0,0.010000,0.000000,0.01,0.01,0.01,0.0


In [174]:
z = downtown_grouped[downtown_grouped['Neighborhood'] == 'Central Bay Street'].T.reset_index()

In [175]:
z.columns = ['Venue', 'Frequency']

In [176]:
z = z.iloc[1:,]

In [177]:
z = z.sort_values('Frequency' , ascending=False).reset_index(drop=True)

In [178]:
def split(row):
    try:
        category_list = row['Venue']
    except:
        category_list = row['venue.categories']
    if len(category_list) == 0:
        return None
    else:
        return (category_list.split('_')[1])


In [179]:
z.head()

,Venue,Frequency
0,Coffee Shop,0.08
1,Clothing Store,0.07
2,Cosmetics Shop,0.04
3,Plaza,0.03
4,Tea Room,0.03


In [180]:
downtown_grouped.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide,King,Richmond",0.0,0.0,0.030000,0.010000,0.00,0.0,0.030000,0.0,0.000000,...,0.0,0.0,0.00,0.0,0.010000,0.000000,0.00,0.00,0.01,0.0
1,Berczy Park,0.0,0.0,0.000000,0.016129,0.00,0.0,0.000000,0.0,0.016129,...,0.0,0.0,0.00,0.0,0.016129,0.000000,0.00,0.00,0.00,0.0
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.0,0.0,0.014706,0.000000,0.00,0.0,0.014706,0.0,0.000000,...,0.0,0.0,0.00,0.0,0.000000,0.014706,0.00,0.00,0.00,0.0
3,"Cabbagetown,St. James Town",0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.00,0.0,0.000000,0.000000,0.00,0.00,0.00,0.0
4,Central Bay Street,0.0,0.0,0.010000,0.010000,0.01,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.01,0.0,0.010000,0.000000,0.01,0.01,0.01,0.0


## Return common venues from Neighborhood

In [181]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [193]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Café,Coffee Shop,Hotel,Burger Joint,Bar,Restaurant,Steakhouse,Asian Restaurant,Gastropub,Deli / Bodega
1,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Seafood Restaurant,Steakhouse,Café,Farmers Market,Cheese Shop,Bakery,Breakfast Spot
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Coffee Shop,Italian Restaurant,Pub,Restaurant,Café,Gym / Fitness Center,Park,Sandwich Place,Bar,Bakery
3,"Cabbagetown,St. James Town",Coffee Shop,Restaurant,Park,Café,Bakery,Italian Restaurant,Pizza Place,Farmers Market,Farm,Butcher
4,Central Bay Street,Coffee Shop,Clothing Store,Cosmetics Shop,Tea Room,Plaza,Sushi Restaurant,Middle Eastern Restaurant,Fast Food Restaurant,Sporting Goods Shop,Chinese Restaurant


# Clustering

In [183]:
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 4, 1, 1, 1], dtype=int32)

In [184]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.681940,-79.378474,3,Playground,Park,Bank,Tennis Court,Building,Donut Shop,Farmers Market,Farm,Falafel Restaurant,Exhibit
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.668160,-79.366602,1,Coffee Shop,Restaurant,Park,Café,Bakery,Italian Restaurant,Pizza Place,Farmers Market,Farm,Butcher
2,M4Y,Downtown Toronto,Church and Wellesley,43.666585,-79.381302,1,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Dance Studio,Gastropub,Fast Food Restaurant,Pub,Café
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.655120,-79.362640,0,Coffee Shop,Breakfast Spot,Restaurant,Spa,Thai Restaurant,Event Space,Mexican Restaurant,Electronics Store,Food Truck,Pub
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657363,-79.378180,1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Bar,Thai Restaurant,Japanese Restaurant,Italian Restaurant,Tea Room


In [185]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [186]:
map_clusters = folium.Map(location=[down_lat, down_lng], zoom_start=10)

## Maping clusters on Map

In [188]:

map_clusters = folium.Map(location=[down_lat, down_lng], zoom_start=10)


# add markers to the map

markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       

In [189]:
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [190]:
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)



# Finish